In [ ]:
import xarray as xr
from emit_tools import emit_xarray
import os
from fsspec.implementations.http import HTTPFileSystem
import holoviews as hv
import hvplot.xarray  # noqa: F401
import numpy as np

from utils import band_index, gamma_adjust, get_rgb_dataset

In [ ]:
# Generate a user token and set it as an environment variable
# Tokens can be generated at https://urs.earthdata.nasa.gov/users/<username>/user_tokens
token = os.environ.get('EARTHDATA_TOKEN')
if token is None:
    raise ValueError('Please set the EARTHDATA_TOKEN environment variable')

In [ ]:
%%time
# Loading data can take around 4 minutes on a 100Mbps connection

# kernel = "EMIT_L2A_RFL_001_20230123T004529_2302216_003" # Hobart
kernel = "EMIT_L2A_RFL_001_20230316T045211_2307503_006" # Canberra

s3_url = "s3://lp-prod-protected/EMITL2ARFL.001/" + kernel + "/" + kernel + ".nc"
s3_url = s3_url.replace("s3://", "https://data.lpdaac.earthdatacloud.nasa.gov/")

fs = HTTPFileSystem(headers={
    "Authorization": f"bearer {token}"
})
ds = emit_xarray(fs.open(s3_url))

# Clean up empty bands. todo: remove the empty bands after.
ds = ds.fillna(np.nan).where(ds.reflectance!=-0.01)
ds

In [ ]:
%%capture --no-stdout

# Select wavelengths to be displayed
r = 2100
g = 950
b = 490

# select brightness - range between 0-1, higher values 'brighten' the whole scene
brightness = 0.4

## End configuration area ##

# Get data structured for rendering
ds_rgb = get_rgb_dataset(ds, [r, g, b], brightness)


In [ ]:
from holoviews import opts
from holoviews.plotting.links import DataLink

POINT_LIMIT = 20
color_cycle = hv.Cycle('Category20')
colors = [color_cycle[i] for i in range(5)]

# RGB image/map
map = ds_rgb.hvplot.rgb(
    x="longitude", y="latitude", bands="bands", aspect="equal", frame_width=600
)

# Set up a holoviews points array to enable plotting of the clicked points
xmid = ds.longitude.values[int(len(ds.longitude) / 2)]
ymid = ds.latitude.values[int(len(ds.latitude) / 2)]

first_point = ([xmid], [ymid], [0])
points = hv.Points(first_point, vdims='id')
points_stream = hv.streams.PointDraw(
    data=points.columns(),
    source=points,
    drag=True,
    num_objects=POINT_LIMIT,
    styles={'fill_color': color_cycle.values[0:POINT_LIMIT], 'line_color': 'gray'}
)

clickxy = hv.streams.Tap(source=map, x=147.5, y=-42.7)

# Function to build spectral plot of clicked location to show on hover stream plot
def click_spectra(data):
    coordinates = []
    if data is None or not any(len(d) for d in data.values()):
        coordinates.append(clicked_points[0][0], clicked_points[1][0])
    else:
        coordinates = [c for c in zip(data['x'], data['y'])]
    
    plots = []
    for i, coords in enumerate(coordinates):
        x, y = coords
        data = ds.sel(longitude=x, latitude=y, method="nearest")
        plots.append(
            data.hvplot.line(
                y="reflectance",
                x="wavelengths",
                color=color_cycle,
                label=f"{i}"
            )
        )
        points_stream.data["id"][i] = i
    return hv.Overlay(plots)

# def update_labels(data):
#     label_points = None
#     if data is not None:
#         label_data = (list(data["x"]), list(data["y"]), [i for i in range(len(data["x"]))])
#         label_points = hv.Points(label_data, vdims='id')
#     return hv.Labels(label_points)

# Define the Dynamic Maps
click_dmap = hv.DynamicMap(click_spectra, streams=[points_stream])
# label_dmap = hv.DynamicMap(update_labels, streams=[points_stream])

# Plot the Map and Dynamic Map side by side
hv.Layout(map * points + click_dmap).cols(1).opts(
    opts.Points(active_tools=['point_draw'], size=10, tools=['hover'], color='white', line_color='gray'),
    opts.Labels(xoffset=0.01, yoffset=0.01, bgcolor='gray', text_color='black'),
    opts.Overlay(show_legend=False, show_title=False, frame_width=600)
)

In [ ]:
import csv 

data = points_stream.data
wavelengths = ds.wavelengths.values

rows = [["id", "x", "y"] + [str(i) for i in wavelengths]]

for p in zip(data['x'], data['y'], data['id']):
    x, y, i = p
    spectra = ds.sel(longitude=x, latitude=y, method="nearest").reflectance.values
    row = [i, x, y] + list(spectra)
    rows.append(row)
    
len(rows)
     

In [ ]:
with open('data.csv', 'w', newline='') as f:
    writer = csv.writer(f)
    writer.writerows(rows)